In [2]:
# MongoDB dependencies & setup

import pymongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# BeautifulSoup dependecies

from bs4 import BeautifulSoup as bs

# request and json dependencies
import requests
import json
from pprint import pprint

import pandas as pd

from flask import Flask, render_template, redirect

from splinter import Browser


# Get Mars News

In [182]:
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

mars_news_url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(mars_news_url)

soup = bs(browser.html, 'html.parser')

news_title = soup.find('div', class_='content_title').text
news_overview= soup.find('div', class_='article_teaser_body').text

print(news_title)
print(news_overview)


Curiosity Tastes First Sample in 'Clay-Bearing Unit'
This new region on Mars might reveal more about the role of water on Mount Sharp.


# Get JPL Featured Mars Image

In [181]:
browser = Browser('chrome', **executable_path, headless=True)

jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)

browser.click_link_by_partial_text('FULL IMAGE')

soup = bs(browser.html, 'html.parser')
mars_image = soup.find('article')


mars_image_url = "https://www.jpl.nasa.gov/spaceimages" + mars_image['style'].split('spaceimages')[1].split("');")[0]
print(mars_image_url)


https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18906-1920x1200.jpg


# Get Mars Weather


In [186]:
mars_weather_url = 'https://twitter.com/marswxreport?lang=en'

html = requests.get(mars_weather_url).text

soup = bs(html, 'html.parser')

mars_weather = soup.find('div', class_='js-tweet-text-container').p.text
print(mars_weather)

InSight sol 135 (2019-04-13) low -96.5ºC (-141.8ºF) high -16.6ºC (2.2ºF)
winds from the SW at 4.2 m/s (9.4 mph) gusting to 11.3 m/s (25.3 mph)
pressure at 7.30 hPapic.twitter.com/bRsLlzn4M0
